In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import csv

In [ ]:
method = 'DBERT'

In [ ]:
lista = []
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/Resultados/'+method+'/Fold1/')
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
  if item.is_file():
    lista.append(item.name)

In [ ]:
for base in lista:
  dic = {}
  pre = {}
  rev= {}
  time = {}
  for i in range(1,11):
    arq = open('/content/drive/My Drive/USP/qualis/Experimentos/Resultados/'+method+'/Fold'+ str(i)+'/' + base, 'r')
    mat_embedding = list()
    reader = csv.reader(arq, delimiter=';')
    entrou = False
    for line in reader:
      if entrou:
        if line[0] in dic:
          dic[line[0]] += float(line[3]) 
          pre[line[0]] += float(line[1])
          rev[line[0]] += float(line[2])
          time[line[0]] += float(line[4])
        else:
          dic[line[0]] = float(line[3]) 
          pre[line[0]] = float(line[1])
          rev[line[0]] = float(line[2])
          time[line[0]] = float(line[4])
      entrou = True

  for model in dic.keys():
    dic[model] = dic[model]/10
    pre[model] = pre[model]/10
    rev[model] = rev[model]/10
    time[model] = time[model]/10

  arq1 = open('/content/drive/My Drive/USP/qualis/Experimentos/Resultados/Resultados-Totais/' + base,'w')
  arq1.write('Parameters;Precisao;Revocacao;F1-Score;Tempo\n')
  for model in dic.keys():
    arq1.write(model + ';' +str("%.4f" %pre[model]) + ';' + ("%.4f" %rev[model]) + ';' +("%.4f" %dic[model]) + ';' +str("%.4f" %time[model]) + '\n')
  arq1.close()

In [ ]:
dic_SVAE = {}
dic_DistilBERT = {}

basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/Resultados/Resultados-Totais/')
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
  if item.is_file():
    caminho = '/content/drive/My Drive/USP/qualis/Experimentos/Resultados/Resultados-Totais/' + item.name 

    df = pd.read_csv(caminho, sep=';')

    best_f1 = max(df['F1-Score'])

    if item.name.split('_')[1] == 'VAE' : dic_SVAE[item.name.split('_')[0]] = best_f1

    if item.name.split('_')[1] == 'DistilBERTML' : dic_DistilBERT[item.name.split('_')[0]] = best_f1

In [ ]:
lista = list()
for base in dic_SVAE.keys():
  lista.append((base, dic_SVAE[base] - dic_DistilBERT[base]))

In [ ]:
lista.sort(key=lambda x: x[1], reverse= True)

In [ ]:
lista

In [ ]:
arquivo = open('/content/drive/My Drive/USP/qualis/Experimentos/Resultados/Resultado-Final.csv', 'w')

arquivo.write('Bases;VAE;DistilBERT;Diference\n')

for b in lista:
  base = b[0]
  tring = base + ';' + str(dic_SVAE[base]) + ';' + str(dic_DistilBERT[base]) + ';' + str("%.4f" % (dic_SVAE[base] - dic_DistilBERT[base])) + '\n'
  arquivo.write(tring)
arquivo.close()

In [ ]:
from scipy.stats import wilcoxon

SVAE = []

DBERT = []

for base in dic_SVAE.keys():
  SVAE.append(dic_SVAE[base])
  DBERT.append(dic_DistilBERT[base])

stat,p = wilcoxon(SVAE,DBERT)

alpha = 0.05

print(p)

if p > alpha:
  print('Mesma distribuição')
else:
  print('Distribuição Diferente')

4.103211601175369e-31
Distribuição Diferente


In [ ]:
D = []
for base in dic_SVAE.keys():
  D.append(abs(dic_SVAE[base] - dic_DistilBERT[base]))

In [ ]:
D = np.array(D)

mean = np.mean(D)

std = np.std(D)

print(mean,std)

0.029527868852459022 0.01413748802260705


In [ ]:
C = mean + (2 * std)
C

0.05780284489767312

In [ ]:
# Code to extract the precision and recall to plot in a Figure

arquivo1 = open('VAE_pre-rev.csv', 'w')
arquivo2 = open('DBERT_pre-rev.csv', 'w')
arquivo1.write('Precisao;Revocacao\n')
arquivo2.write('Precisao;Revocacao\n')

basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/Resultados/Resultados-Totais/')
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
  if item.is_file():
    caminho = '/content/drive/My Drive/USP/qualis/Experimentos/Resultados/Resultados-Totais/' + item.name  

    df = pd.read_csv(caminho, sep=';')

    best_f1 = max(df['F1-Score'])

    pre = df[df['F1-Score']==best_f1]['Precisao']

    rev = df[df['F1-Score']==best_f1]['Revocacao']
      
    if item.name.split('_')[1] == 'VAE' :
      arquivo1.write(str("%.4f" %pre.iloc[0]) + ';' + str("%.4f" %rev.iloc[0]) + '\n')

    if item.name.split('_')[1] == 'DistilBERTML' : 
      arquivo2.write(str("%.4f" %pre.iloc[0]) + ';' + str("%.4f" %rev.iloc[0]) + '\n')

arquivo1.close()
arquivo2.close()